In [13]:
import pandas as pd
import pdfplumber
import camelot

MAX_Y = 792
filename = '8.18.20_COVID-19_Update_FINAL-1.pdf'

with pdfplumber.open(filename) as pdf:

    for i, page in enumerate(pdf.pages):
        if "Project Roomkey Locations:" in page.extract_text():
            start_page = page
        if "Overall Totals:" in page.extract_text():
            end_page = page

    try:
        print("Start page:", start_page.page_number)
        print("End page:", end_page.page_number)
    except:
        print("Error: couldn't find project roomkey data :(")

    start_page_words = start_page.extract_words()
    words = list(map(lambda x: x.get('text'), start_page.extract_words()))

    for i, word in enumerate(start_page_words):
        if (words[i] == "Project" and words[i + 1] == "Roomkey" and words[i + 2] == "Locations:"):
            x_start = start_page_words[i + 3].get('x0') - 5
            y_start = MAX_Y - start_page_words[i + 3].get('top')

    y_page_end = MAX_Y - (start_page_words[-1].get('top') - 18)

    end_page_words = end_page.extract_words()
    words = list(map(lambda x: x.get('text'), end_page.extract_words()))

    y_page_start = MAX_Y - end_page_words[0].get('top')

    for i, word in enumerate(end_page_words):
        if (words[i] == "Overall" and words[i + 1] == "Totals:"):
            x_end = end_page_words[i + 4].get('x1') + 27
            y_end = MAX_Y - end_page_words[i + 4].get('bottom')

    labels = ["x_start", "x_end", "y_start", "y_end", "y_page_start", "y_page_end"]
    coords = dict(zip(labels, (x_start, x_end, y_start, y_end, y_page_start, y_page_end)))

    try:
        print(coords)
    except:
        print("Error: couldn't get table coords :(")



    """
    x of table beginning ("Locations:" + 1)
    x of table end ("Clients" + 1, "Overall Totals" + 3)
    y of table beginning ("Locations:" + 1)
    y of table end ("Overall Totals", "Overall Totals" + 3)
    
    y of new page beginning (first word)
    y of new page end (5th to last word)
    """

    all_data = []

    # For each page containing tables of data...
    for page_number in range(start_page.page_number, end_page.page_number + 1):
        # Get (x,y) coordinates for top left and bottom right corner of the table
        x0 = x_start
        y0 = MAX_Y - (y_start if page_number == start_page.page_number else y_page_start)
        x1 = x_end
        y1 = MAX_Y - (y_end if page_number == end_page.page_number else y_page_end)
    
        # Save a cropped image of the table
        cropped = pdf.pages[page_number - 1].crop((x0, y0, x1, y1))
        img = cropped.to_image()
        img.save(str(page_number) + '_' + filename[:-4] + '.png', format="PNG")

        # Read the table's data

        # Process the table data

        
    # Export all table data


    start_page_cropped = start_page.crop((x_start, MAX_Y - y_start, x_end, MAX_Y - y_page_end))
    middle_page_cropped = pdf.pages[4].crop((x_start, MAX_Y - y_page_start, x_end, MAX_Y - y_page_end))
    end_page_cropped = end_page.crop((x_start, MAX_Y - y_page_start, x_end, MAX_Y - y_end))
    
    start_img = start_page_cropped.to_image()
    middle_img = middle_page_cropped.to_image()
    end_img = end_page_cropped.to_image()
    
    start_img.save('0.png', format='PNG')
    middle_img.save('1.png', format='PNG')
    end_img.save('2.png', format='PNG')



    start_page_region = ','.join(str(coord) for coord in [x_start, y_start, x_end, y_page_end])
    middle_page_region = ','.join(str(coord) for coord in [x_start, y_page_start, x_end, y_page_end])
    end_page_region = ','.join(str(coord) for coord in [x_start, y_page_start, x_end, y_end])

    start_table = camelot.read_pdf(filename, pages=str(start_page.page_number), table_regions=[start_page_region], strip_text='\n')
    middle_table = camelot.read_pdf(filename, pages=str(start_page.page_number + 1), table_regions=[middle_page_region], strip_text='\n')
    end_table = camelot.read_pdf(filename, pages=str(end_page.page_number), table_regions=[end_page_region], strip_text='\n')

    all_tables = pd.concat([start_table[0].df, middle_table[0].df, end_table[0].df])

    print(all_tables)

    all_tables.to_csv('all_data.csv')



Start page: 4
End page: 6
{'x_start': Decimal('157.380'), 'x_end': Decimal('539.188'), 'y_start': Decimal('135.860'), 'y_end': Decimal('312.170'), 'y_page_start': Decimal('736.680'), 'y_page_end': Decimal('61.080')}
                                    0            1                 2  \
0         Service Planning Area (SPA)  Total Rooms  Date Operational   
1        SPA 1 – Antelope Valley (#1)           94          4/6/2020   
0              SPA 8 – South Bay (#1)           60          4/8/2020   
1   SPA 2 – San Fernando Valley (# 1)           52          4/5/2020   
2       SPA 3 San Gabriel Valley (#1)           49          4/6/2020   
3    SPA 2 – San Fernando Valley (#2)           74          4/5/2020   
4    SPA 2 – San Fernando Valley (#3)           50         4/11/2020   
5                   SPA 5 – West (#1)          136          4/3/2020   
6                   SPA 7 – East (#1)           50         4/17/2020   
7     SPA 3 – San Gabriel Valley (#2)           87         4/12/